In [ ]:
!pip install sagemaker dill -U 

In [ ]:
from typing import Final
import sagemaker, dill, os, glob
import pandas as pd
import numpy as np
bucket: Final[str] = sagemaker.session.Session().default_bucket()
roke: Final[str] = sagemaker.get_execution_role()
sess = sagemaker.session.Session()

In [ ]:
s3_base_uri = sess.upload_data('./bench_data/',key_prefix='bench_data')

In [ ]:
!rm -rf bench_src
!mkdir -p bench_src

In [ ]:
%%writefile bench_src/requirements.txt
rrcf==0.4.3
dill==0.3.4
matplotlib==3.5.3

## シングルプロセス

In [ ]:
%%writefile src/train.py
import rrcf
import os, glob
import pandas as pd
import numpy as np
import dill
import json
from matplotlib import pyplot as plt
from scipy import stats
from typing import Final
import logging
import sys
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

# ハイパーパラメータ取得
def get_hps():
    logger.debug('setting hyperparameters...')
    hps: Final[dict] = json.loads(os.environ.get('SM_HPS'))
    # 設定されていなかった場合のデフォルト値
    hps.setdefault('num_trees', 50)
    hps.setdefault('shingle_size', 3)
    hps.setdefault('tree_size', 512)
    logger.debug('got hyperparameters...')
    logger.info('hps is ...')
    logger.info(hps)
    return hps

# csvファイル群を連結して DataFrame にする
def load_csv_files(csv_dir):
    logger.debug('loading csv...')
#     csv_list = sorted(glob.glob(os.path.join(csv_dir,'*.csv')))
#     df_list = [pd.read_csv(csv_file, header=None) for csv_file in csv_list]
#     df = pd.concat(df_list, ignore_index=True)
    csv_file = glob.glob(os.path.join(csv_dir,'*.csv'))[0]
    df = pd.read_csv(csv_file, header=None)
    logger.debug('loaded csv')
    return df

# 異常スコアを計算する
# 詳細 : https://klabum.github.io/rrcf/taxi.html
# shingle 済のデータに異常スコアを付与したDataFrameを返す
def calc_score(df,hps):
    # RCF 準備
    logger.debug('preparing RCF...')
    data = df[0].astype(float).values
    points = rrcf.shingle(data, size=hps['shingle_size'])
    points = np.vstack([point for point in points])
    n = points.shape[0]
    sample_size_range = (n // hps['tree_size'], hps['tree_size'])
    logger.debug('prepared RCF')
    
    # RCF を生成
    logger.debug('generating RCF...')
    forest = []
    while len(forest) < hps['num_trees']:
        ixs = np.random.choice(n, size=sample_size_range,
                               replace=False)
        trees = [rrcf.RCTree(points[ix], index_labels=ix) for ix in ixs]
        forest.extend(trees)
    logger.debug('generated RCF')
    
    # 異常スコア算出
    logger.debug('calculating score...')
    avg_codisp = pd.Series(0.0, index=np.arange(n))
    index = np.zeros(n)
    for tree in forest:
        codisp = pd.Series({leaf : tree.codisp(leaf) for leaf in tree.leaves})
        avg_codisp[codisp.index] += codisp
        np.add.at(index, codisp.index.values, 1)
    avg_codisp /= index
    logger.debug('calculated score')
    
    # result の整理
    logger.debug('organizing score...')
    columns = [i for i in range(points.shape[1])]
    result_df = pd.DataFrame(points, columns=columns, dtype='float')
    result_df['score'] = pd.Series(avg_codisp)
    result_df['scaled_score'] = result_df['score']/result_df['score'].max()
    
    logger.debug('organized score')
    return result_df, forest

# 異常判定
def calc_threshold(df):
    logger.debug('calculating calc_threshold...')
    df['zscore'] = stats.zscore(df['score'])
    df['anomaly'] = df['zscore'].apply(lambda x: True if x>3 else False)
    return df

# グラフの描画
def draw_graph(result_df):
    logger.debug('drawing graph...')
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(3,1,1)
    for col in result_df.columns.values:
        if type(col)==int:
            ax1.plot(result_df[col])
        else:
            break
    ax2 = fig.add_subplot(3,1,2)
    ax2.plot(result_df['score'])
    ax3 = fig.add_subplot(3,1,3)
    ax3.plot(result_df['anomaly'])
    logger.debug('drawn graph')
    return fig

def main(model_base_dir, target_dir_list, hps):
    for target_dir in target_dir_list:
        logger.info(f"target : {target_dir.replace('/opt/ml/input/data/training/','')}")
        
        # 生成物の出力先設定
        artifact_dir = os.path.join(model_base_dir,target_dir.replace('/opt/ml/input/data/training/',''))
        os.makedirs(artifact_dir)
#         model_path = os.path.join(artifact_dir,'model.dill')
#         graph_path = os.path.join(artifact_dir,'graph.png')
        csv_path = os.path.join(artifact_dir,'result.csv')
        
        # 使用するデータをロードして１つの DataFrame にまとめる
        df = load_csv_files(target_dir)
        
        # スコアの算出と算出に使用した shingle 済データと、RCF を作成
        result_df, forest = calc_score(df, hps)
        
        # しきい値計算と判定
        result_df = calc_threshold(result_df)
        
        # 結果 DF の出力
        result_df.to_csv(csv_path,index=False)
        
#         # RCF の出力
#         with open(model_path,'wb') as f:
#             dill.dump(forest, f)
        
#         # shingle 済データとスコアのグラフ描画と出力
#         fig = draw_graph(result_df)
#         fig.savefig(graph_path,dpi=300)

if __name__ == '__main__':
    logger.info('exec start...')
    hps = get_hps()
    input_dir: Final[str] = os.environ.get('SM_CHANNEL_TRAINING')
    model_base_dir: Final[str] = os.environ.get('SM_MODEL_DIR')
    target_dir_list: Final[list] = glob.glob(os.path.join(input_dir,'*'))
    main(model_base_dir, target_dir_list, hps)
    logger.info('completed')
    exit()

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    base_job_name='rrcf-single-process',
    entry_point='train.py',
    source_dir = './src/',
    py_version='py3', 
    framework_version='1.0-1',
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'num_trees': 128,
        'shingle_size': 10,
        'tree_size': 1024,
    },
    volume_size=200
)
estimator.fit(
    s3_base_uri
)

## マルチプロセス

In [ ]:
!rm -rf bench_multi_src
!mkdir -p bench_multi_src

In [ ]:
%%writefile bench_multi_src/requirements.txt
rrcf==0.4.3
dill==0.3.4
matplotlib==3.5.3

In [ ]:
%%writefile bench_multi_src/train_multi_processing.py
import rrcf
import os, glob
import pandas as pd
import numpy as np
import dill
import json
from matplotlib import pyplot as plt
from scipy import stats
from typing import Final
import logging
import sys
from multiprocessing import Pool

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

input_dir: Final[str] = os.environ.get('SM_CHANNEL_TRAINING')
model_base_dir: Final[str] = os.environ.get('SM_MODEL_DIR')

logger.debug('setting hyperparameters...')
hps: Final[dict] = json.loads(os.environ.get('SM_HPS'))
# 設定されていなかった場合のデフォルト値
hps.setdefault('num_trees', 50)
hps.setdefault('shingle_size', 3)
hps.setdefault('tree_size', 512)
hps.setdefault('process_num',1)
logger.debug('got hyperparameters...')
logger.info('hps is ...')
logger.info(hps)
target_dir_list: Final[list] = glob.glob(os.path.join(input_dir,'*'))

# csvファイル群を連結して DataFrame にする
def load_csv_files(csv_dir):
    logger.debug('loading csv...')
#     csv_list = sorted(glob.glob(os.path.join(csv_dir,'*.csv')))
#     df_list = [pd.read_csv(csv_file, header=None) for csv_file in csv_list]
#     df = pd.concat(df_list, ignore_index=True)
    csv_file = glob.glob(os.path.join(csv_dir,'*.csv'))[0]
    df = pd.read_csv(csv_file, header=None)
    logger.debug('loaded csv')
    return df

# 異常スコアを計算する
# 詳細 : https://klabum.github.io/rrcf/taxi.html
# shingle 済のデータに異常スコアを付与したDataFrameを返す
def calc_score(df,hps):
    # RCF 準備
    logger.debug('preparing RCF...')
    data = df[0].astype(float).values
    points = rrcf.shingle(data, size=hps['shingle_size'])
    points = np.vstack([point for point in points])
    n = points.shape[0]
    sample_size_range = (n // hps['tree_size'], hps['tree_size'])
    logger.debug('prepared RCF')
    
    # RCF を生成
    logger.debug('generating RCF...')
    forest = []
    while len(forest) < hps['num_trees']:
        ixs = np.random.choice(n, size=sample_size_range,
                               replace=False)
        trees = [rrcf.RCTree(points[ix], index_labels=ix) for ix in ixs]
        forest.extend(trees)
    logger.debug('generated RCF')
    
    # 異常スコア算出
    logger.debug('calculating score...')
    avg_codisp = pd.Series(0.0, index=np.arange(n))
    index = np.zeros(n)
    for tree in forest:
        codisp = pd.Series({leaf : tree.codisp(leaf) for leaf in tree.leaves})
        avg_codisp[codisp.index] += codisp
        np.add.at(index, codisp.index.values, 1)
    avg_codisp /= index
    logger.debug('calculated score')
    
    # result の整理
    logger.debug('organizing score...')
    columns = [i for i in range(points.shape[1])]
    result_df = pd.DataFrame(points, columns=columns, dtype='float')
    result_df['score'] = pd.Series(avg_codisp)
    result_df['scaled_score'] = result_df['score']/result_df['score'].max()
    
    logger.debug('organized score')
    return result_df, forest

# 異常判定
def calc_threshold(df):
    logger.debug('calculating calc_threshold...')
    df['zscore'] = stats.zscore(df['score'])
    df['anomaly'] = df['zscore'].apply(lambda x: True if x>3 else False)
    return df

# グラフの描画
def draw_graph(result_df):
    logger.debug('drawing graph...')
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(3,1,1)
    for col in result_df.columns.values:
        if type(col)==int:
            ax1.plot(result_df[col])
        else:
            break
    ax2 = fig.add_subplot(3,1,2)
    ax2.plot(result_df['score'])
    ax3 = fig.add_subplot(3,1,3)
    ax3.plot(result_df['anomaly'])
    logger.debug('drawn graph')
    return fig

def main(target_dir):
    logger.info(f"target : {target_dir.replace('/opt/ml/input/data/training/','')}")
    # 生成物の出力先設定
    artifact_dir = os.path.join(model_base_dir,target_dir.replace('/opt/ml/input/data/training/',''))
    os.makedirs(artifact_dir)
#         model_path = os.path.join(artifact_dir,'model.dill')
#         graph_path = os.path.join(artifact_dir,'graph.png')
    csv_path = os.path.join(artifact_dir,'result.csv')

    # 使用するデータをロードして１つの DataFrame にまとめる
    df = load_csv_files(target_dir)

    # スコアの算出と算出に使用した shingle 済データと、RCF を作成
    result_df, forest = calc_score(df, hps)

    # しきい値計算と判定
    result_df = calc_threshold(result_df)

    # 結果 DF の出力
    result_df.to_csv(csv_path,index=False)
        
#         # RCF の出力
#         with open(model_path,'wb') as f:
#             dill.dump(forest, f)
        
#         # shingle 済データとスコアのグラフ描画と出力
#         fig = draw_graph(result_df)
#         fig.savefig(graph_path,dpi=300)

if __name__ == '__main__':
    logger.info('exec start...')
    with Pool(processes=hps['process_num']) as pool:
        pool.map(main,target_dir_list)
    logger.info('completed')
    exit()

### c5.xlarge 3 processes

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    base_job_name='rrcf-3-processes',
    entry_point='train_multi_processing.py',
    source_dir = './bench_multi_src/',
    py_version='py3', 
    framework_version='1.0-1',
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'num_trees': 128,
        'shingle_size': 10,
        'tree_size': 1024,
        'process_num':3
    },
    volume_size=200
)
estimator.fit(
    s3_base_uri
)

## c5.xlarge 4 processes

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    base_job_name='rrcf-4-processes',
    entry_point='train_multi_processing.py',
    source_dir = './bench_multi_src/',
    py_version='py3', 
    framework_version='1.0-1',
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'num_trees': 128,
        'shingle_size': 10,
        'tree_size': 1024,
        'process_num':4
    },
    volume_size=200
)
estimator.fit(
    s3_base_uri
)

### c5.4xlarge 15 processes

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    base_job_name='rrcf-15-processes',
    entry_point='train_multi_processing.py',
    source_dir = './bench_multi_src/',
    py_version='py3', 
    framework_version='1.0-1',
    instance_count=1,
    instance_type='ml.c5.4xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'num_trees': 128,
        'shingle_size': 10,
        'tree_size': 1024,
        'process_num':15
    },
    volume_size=200
)
estimator.fit(
    s3_base_uri
)

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    base_job_name='rrcf-16-processes',
    entry_point='train_multi_processing.py',
    source_dir = './bench_multi_src/',
    py_version='py3', 
    framework_version='1.0-1',
    instance_count=1,
    instance_type='ml.c5.4xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'num_trees': 128,
        'shingle_size': 10,
        'tree_size': 1024,
        'process_num':16
    },
    volume_size=200
)
estimator.fit(
    s3_base_uri
)

### c5.9xlarge 35 parallels

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    base_job_name='rrcf-35-processes',
    entry_point='train_multi_processing.py',
    source_dir = './bench_multi_src/',
    py_version='py3', 
    framework_version='1.0-1',
    instance_count=1,
    instance_type='ml.c5.9xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'num_trees': 128,
        'shingle_size': 10,
        'tree_size': 1024,
        'process_num':35
    },
    volume_size=200
)
estimator.fit(
    s3_base_uri
)

### c5.9xlarge 36 parallels

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    base_job_name='rrcf-36-processes',
    entry_point='train_multi_processing.py',
    source_dir = './bench_multi_src/',
    py_version='py3', 
    framework_version='1.0-1',
    instance_count=1,
    instance_type='ml.c5.9xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'num_trees': 128,
        'shingle_size': 10,
        'tree_size': 1024,
        'process_num':36
    },
    volume_size=200
)
estimator.fit(
    s3_base_uri
)

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    base_job_name='rrcf-71-processes',
    entry_point='train_multi_processing.py',
    source_dir = './bench_multi_src/',
    py_version='py3', 
    framework_version='1.0-1',
    instance_count=1,
    instance_type='ml.c5.18xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'num_trees': 128,
        'shingle_size': 10,
        'tree_size': 1024,
        'process_num':71
    },
    volume_size=200
)
estimator.fit(
    s3_base_uri
)